In [ ]:
import numpy as np

G = 6.67430e-11  # gravitational constant

class Body:
    def __init__(self, name, mass, position, velocity):
        self.name = name
        self.mass = mass
        self.position = np.array(position)
        self.velocity = np.array(velocity)

# force calculator class with three children, brute force, barnes-hut, and fast multipole
class ForceCalculator:
    def compute_accelerations(self, bodies):
        pass
    
class DirectForceCalculator(ForceCalculator):
    
    def compute_accelerations(self, bodies):
        positions = np.array([b.position for b in bodies])
        masses = np.array([b.mass for b in bodies])
        accelerations = np.zeros_like(positions)

        for i in range(len(bodies)):
            r_vec = positions - positions[i]
            r = np.linalg.norm(r_vec, axis=1)
            mask = r > 0
            accelerations[i] = np.sum(
                G * masses[mask, None] * r_vec[mask] / r[mask, None]**3,
                axis=0
            )
        return accelerations

class BarnesHutForceCalculator(ForceCalculator):

    def compute_accelerations(self, bodies):
        # TBI

class FastMultipoleForceCalculator(ForceCalculator):

    def compute_accelerations(self, bodies):
        #TBI

# integrator class with two children, RK4 and Leapfrog (symplectic)
class Integrator():
    
    def step(self, bodies, h): 
        pass

class RK4Integrator(Integrator):
    
    def __init__(self, force_calculator):
        self.force_calculator = force_calculator

    def step(self, bodies, dt):
        # concatenate positions and velocities of bodies
        p0 = np.array([b.position for b in bodies])
        v0 = np.array([b.velocity for b in bodies])

        def derivatives(positions, velocities):
            # Temporarily update positions to compute accelerations
            for i, b in enumerate(bodies):
                b.position = positions[i]
                b.velocity = velocities[i]
            accelerations = self.force_calculator.compute_accelerations(bodies)
            return velocities, accelerations

        k1v, k1a = derivatives(y0_pos, y0_vel)
        k2v, k2a = derivatives(y0_pos + 0.5*dt*k1v, y0_vel + 0.5*dt*k1a)
        k3v, k3a = derivatives(y0_pos + 0.5*dt*k2v, y0_vel + 0.5*dt*k2a)
        k4v, k4a = derivatives(y0_pos + dt*k3v, y0_vel + dt*k3a)

        new_positions = y0_pos + (dt/6)*(k1v + 2*k2v + 2*k3v + k4v)
        new_velocities = y0_vel + (dt/6)*(k1a + 2*k2a + 2*k3a + k4a)

        for i, b in enumerate(bodies):
            b.position = new_positions[i]
            b.velocity = new_velocities[i]

class LeapfrogIntegrator(Integrator):
    
    def __init__(self, force_calculator):
        self.force_calculator = force_calculator
    
    def step(self, bodies, dt):
        pass

# simulation
class NBodySimulation:
    def __init__(self, bodies, method=brute_force):
        self.bodies = bodies
        self.integrator = RK4Integrator(method)

    def run(self, dt, steps, record=True):
        if not record:
            for _ in range(steps):
                self.integrator.step(self.bodies, dt)
            return

        trajectories = {b.name: [b.position.copy()] for b in self.bodies}
        for _ in range(steps):
            self.integrator.step(self.bodies, dt)
            for b in self.bodies:
                trajectories[b.name].append(b.position.copy())
        return trajectories

In [ ]:
sun = Body("Sun", 1.98847e30, [0, 0, 0], [0, 0, 0])
earth = Body("Earth", 5.972e24, [1.496e11, 0, 0], [0, 29_780, 0])
    sim = NBodySimulation([sun, earth], method="direct")

    traj = sim.run(dt=60*60*6, steps=1000)

    import matplotlib.pyplot as plt
    for name, path in traj.items():
        path = np.array(path)
        plt.plot(path[:, 0], path[:, 1], label=name)
    plt.legend()
    plt.axis("equal")
    plt.title("N-Body Simulation (RK4)")
    plt.show()